In [5]:
import json, requests, time, datetime, math
from pprint import pprint
import numpy as np
import pandas as pd
from scipy.stats import norm
from pandas_datareader import data as pdr
from functools import reduce
import constants, web3_service, pool_data_service

# Used to find portfolio weights for an array of balances
def getWeights(balances):
    total = 0
    weights = []
    for balance in balances:
        total += balance['liquidity']
    for balance in balances:
        percentage = balance['liquidity']/total
        weights.append(percentage)
    return weights

# Used to find historical USD values for stablecoins
def getCryptoCompareReturns(token):
    result = requests.get(f'https://min-api.cryptocompare.com/data/v2/histoday?fsym={token}&tsym=USD&limit=720')
    json = result.json()
    df = pd.DataFrame([object for object in json['Data']['Data']])
    df.insert(0, 'Date', pd.to_datetime(df['time'],unit='s'))
    df.drop(['high', 'open', 'low', 'volumefrom', 'volumeto', 'conversionType', 'conversionSymbol', 'time'], axis=1, inplace=True)
    df.set_index('Date', inplace=True)
    returns = df.copy().pct_change().fillna(value=0, axis=0).rename(columns={'close': f'daily_returns_{token}'})
    return returns
    
# TODO - Fix Dates
# Used to find historical USD values for all coins but stablecoins      
def getReturns(tokens): 
    first_date = '2017-10-06'
    last_date = '2019-09-25'
    df_list = []
    for token in tokens:
        if (token['token'] == 'wbtc'):
            token = token['token'][1:].upper()
        else:
            token = token['token'].upper()
        if (token == 'DAI' or token == 'USDC' or token == 'MKR' or token == 'TUSD'):
            ticker_returns = getCryptoCompareReturns(token)
            df_list.append(ticker_returns)
        else:
            ticker = f'{token}-USD'
            ticker_close = pdr.get_data_yahoo(ticker, first_date, last_date)[['Close']]
            ticker_returns = ticker_close.copy().pct_change().fillna(value=0, axis=0).rename(columns={'Close': f'daily_returns_{token}'})
            df_list.append(ticker_returns)
    df = reduce(lambda x, y: pd.merge(x, y, on = 'Date'), df_list)
    return df

def value_at_risk(returns, weights, alpha=0.95, lookback_days=520):
    # Multiply asset returns by weights to get one weighted portfolio return
    portfolio_returns = returns.iloc[-lookback_days:].dot(weights)
    # Compute the correct percentile loss and multiply by value invested
    return np.percentile(portfolio_returns, 100 * (1-alpha))

def cvar(returns, weights, alpha=0.95, lookback_days=520):
    var = value_at_risk(returns, weights, alpha, lookback_days=lookback_days)
    returns = returns.fillna(0.0)
    portfolio_returns = returns.iloc[-lookback_days:].dot(weights)
    # Get back to a return rather than an absolute loss
    var_pct_loss = var / 1
    return np.nanmean(portfolio_returns[portfolio_returns < var_pct_loss])

def generate_cvar_from_balances(balances):
    weights = getWeights(balances)
    returns = getReturns(balances)
    returns = returns.fillna(0.0)
    portfolio_returns = returns.fillna(0.0).iloc[-520:].dot(weights)
    portfolio_cvar = cvar(returns, weights, 0.99, 520)
    return portfolio_cvar

def EMACalc(m_array, m_range):
    k = 2/(m_range + 1)
    # first item is just the same as the first item in the input
    ema_array = [m_array[0]]
    # for the rest of the items, they are computed with the previous one
    i = 1
    while i < m_range:
        ema_array.append(m_array[i] * k + ema_array[i - 1] * (1 - k))
        i += 1
    return ema_array[len(ema_array) - 1]

# Normalize value from a list of objects with predefined shape
def normalize_data(val, list):
    max_value = max(list)
    min_value = min(list)
    return (val - min_value) / (max_value - min_value)
    

In [6]:
# Pulling and calculating Compound data
compound_tokens = [x['token'] for x in constants.compoundContractInfo]
compound_balances = [pool_data_service.fetch_data_for_pool('compound', t) for t in compound_tokens]
compound_portfolio_cvar = generate_cvar_from_balances(compound_balances)
# add instead of subtract here because cvar from this function is negative
constants.compound_values['cvar'] = 1 + compound_portfolio_cvar
pprint(constants.compound_values)

{'cvar': 0.8864836611499236,
 'hasBugBounty': 1,
 'insuranceRisk': 0,
 'isCodeAudited': 1,
 'isCodeFormallyVerified': 1,
 'isCodeOpenSource': 1}


In [7]:
# Pulling and calculating dYdX data
dydx_tokens = [x for x in constants.dydxContractInfo['markets']]
dydx_balances = [pool_data_service.fetch_data_for_pool('dydx', t) for t in dydx_tokens]
dydx_portfolio_cvar = generate_cvar_from_balances(dydx_balances)
# add instead of subtract here because cvar from this function is negative
constants.dydx_values['cvar'] = 1 + dydx_portfolio_cvar
pprint(constants.dydx_values)

{'cvar': 0.8657417546227986,
 'hasBugBounty': 1,
 'insuranceRisk': 0,
 'isCodeAudited': 1,
 'isCodeFormallyVerified': 0,
 'isCodeOpenSource': 1}


In [8]:
# Pulling and calculating Fulcrum data
fulcrum_tokens = [x['token'] for x in constants.fulcrumContractInfo]
fulcrum_balances = [pool_data_service.fetch_data_for_pool('fulcrum', t) for t in fulcrum_tokens]
fulcrum_portfolio_cvar = generate_cvar_from_balances(fulcrum_balances)
# add instead of subtract here because cvar from this function is negative
constants.fulcrum_values['cvar'] = 1 + fulcrum_portfolio_cvar
pprint(constants.fulcrum_values)

{'cvar': 0.9484978791928579,
 'hasBugBounty': 1,
 'insuranceRisk': 0,
 'isCodeAudited': 1,
 'isCodeFormallyVerified': 0,
 'isCodeOpenSource': 1}


In [9]:
# Pulling and calculating Nuo data
nuo_tokens = [x['token'] for x in constants.nuoContractInfo]
nuo_balances = [pool_data_service.fetch_data_for_pool('nuo', t) for t in nuo_tokens]
nuo_portfolio_cvar = generate_cvar_from_balances(nuo_balances)
# add instead of subtract here because cvar from this function is negative
constants.nuo_values['cvar'] = 1 + nuo_portfolio_cvar
pprint(constants.nuo_values)

{'cvar': 0.9301027909008657,
 'hasBugBounty': 0,
 'insuranceRisk': 0,
 'isCodeAudited': 0,
 'isCodeFormallyVerified': 0,
 'isCodeOpenSource': 1}


In [17]:
def calculate_score(protocol, token, liquidity_value, collateral_value):
    if protocol == 'dydx':
        protocol_values = constants.dydx_values
    elif protocol == 'compound':
        protocol_values = constants.compound_values
    elif protocol == 'fulcrum':
        protocol_values = constants.fulcrum_values
    else:
        protocol_values = constants.nuo_values
    weights = constants.weights
    score = weights['auditedCode'] * protocol_values['isCodeAudited'] + weights['allCodeOSS'] * protocol_values['isCodeOpenSource'] + weights['formalVer'] * protocol_values['isCodeFormallyVerified'] + weights['hasBugBounty'] * protocol_values['hasBugBounty'] + weights['cVaR'] * protocol_values['cvar'] + weights['poolCollateralization'] * collateral_value + weights['poolLiquidity'] * liquidity_value
    score = round(score, 2) * 10
    score = "{:.1f}".format(score)
    result = {
        'asset': token,
        'protocol': protocol,
        'metrics': {
            'score': score,
            'liquidity': liquidity_value,
            'collateralized': collateral_value
        }
    }
    return result

def calculate_scores():
    # Get all pool data
    all_pool_data = pool_data_service.fetch_data_for_all_pools()
    liquidity_array = [math.log(p['liquidity']) for p in all_pool_data]
    utilization_array = [p['utilizationRate'] for p in all_pool_data]
    results = []
    for data in all_pool_data:
        liquidity_value = normalize_data(math.log(data['liquidity']), liquidity_array)
        # Subtracting from 1 because lower utilization is safer
        utilization_value = 1 - normalize_data(data['utilizationRate'], utilization_array)
        score = calculate_score(data['protocol'], data['token'], liquidity_value, utilization_value)
        results.append(score)
    return results

In [18]:
scores = calculate_scores()
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(scores, f, ensure_ascii=False, indent=4)